In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb
os.chdir(d['py'])
from gmsPython import nestingTree
from valueShares import nestedShares
from mProduction import NestedCES

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb3.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.


# Production module

This notebook serves as a bit of a tutorial for how the CGE model is generally build, solved, and calibrated. Instead of relying on "high-level" functions that take care of things in a single line of code, this goes through more steps and explains each part. The notebook ```mP.ipynb``` contains the high-level codes for doing this in a few lines of code. 

Throughout, we rely on classes  ```gmsWrite, nestingTree, gmsPy.Group, gmsPy.jTerms, gmsPyModels.Model``` (all from ```gmsPython``` package) and database functions from the ```pyDatabases``` package.

*Load data:*

In [2]:
t0 = 2019 # set baseline year
name = 'vA' # specify name of the CGE version we are currently working on
db = GpyDB(os.path.join(d['data'], f'{name}_{t0}')) # Load 
ws = db.ws # run everything from the same ws

### Overview 

The production module consists of four blocks of equations:
1. nested CES functions,
2. a block of equations that specify the adjustment costs and accumulation of durables,
3. a block of equations that maps from regulation to effective prices, and
4. a calibration block of equations that helps adjust the total amount of taxes that the government extracts from production sectors.

Beyond the blocks of equation, the module contains other relevant information and methods that we can apply. This includes:
* *Data generating methods*: Most of the model data is produced in a previous step (```ModelData.ipynb```), but the production module contains methods to generate relevant subsets used e.g. for calibration purposes and produce initial values for variables used in the equations (that are not already defined by the IO data).
* *Grouping of variables*: The variables of the model are divided into four groups (1) always exogenous, (2) always endogenous, (3) endogenous in baseline/exogenous in calibration, (4) exogenous in baseline/endogenous in calibration. These groups are defined using the ```gmsPython.gmsPy.Group``` class. The reason why we do this is that ```Group``` instances have automated writing methods.
* *Specification of models:* The method ```self.models(state = 'B')``` collects the relevant block of equations required to run/solve the baseline ('B') and calibration ('C') models.
* *Writing methods:* All of the methods used to specify groups and equations are ultimately used to automate writing the text for the model.

Before we get to the production module, however, we need to specify the nesting structure using the ```nestingTree``` class below.

## 1. Define CES value shares

To specify the nesting structure of the model, we use a small customized class from ```gmsPython.nestingTree```: We do not need to go into detail on how these works (they are very simple), except to note that we initialize them by adding the nesting structure as a list of tuples (we already have this in our database). These classes automatically produce a number of subsets that specify e.g. what are inputs, intermediate inputs, and outputs.

### 1.1. Nesting trees

As we ultimately want to be able to replace the 'Waste' sector with a more specialized module, we start by splitting up the nesting structure into two objects: One for the waste sector and one for other:

In [3]:
wasteNest = adj.rc_pd(db('nestProduction'), pd.Index(['Waste'], name = 's')) # the adj.rc_pd(s, c) adjusts the symbol 's' by matching with the condition 'c'. It operates by matching the indices.
otherNest = db('nestProduction').difference(wasteNest) 

Next, we initialize the two nesting trees separately (and name them 'W' and 'O' respectively), and finally collect them in one "aggregate" nesting tree that we call "P" for production here:

In [4]:
waste = nestingTree.Tree('W', tree = wasteNest.to_list(), f = 'CES') 
other = nestingTree.Tree('O', tree = otherNest.to_list(), f = 'CES')
nest = nestingTree.AggTree(name = 'P', trees = {t.name: t for t  in (waste, other)}, ws = ws)
nest(namespace = {str(n)+'_input':n for n in db('n')}) # the __call__ method establishes relevant symbols to create the gams model

*Note: The notebook ```nestingTree.ipynb``` under the gmsPython package details what this nesting tree class does in detail.*

The nesting tree class establishes symbols for the aggregate tree, but also for the 'individual' trees (waste, other). The names of the nesting trees are important here, because they are used as prefixes to the local symbol names. For instance, the mapping from the aggregate tree and the individual ones are stored as:

In [5]:
nest.db('P_map'), nest.db('O_map'), nest.db('W_map'); # syntax to access the different symbols

We keep track of the symbol names using a simple namespace dictionary (```self.ns```) that maps from the global name (e.g. 'input') to the specific name (e.g. 'P_input'). The methods ```self.n``` and ```self.get``` allows us to access names and symbols using the global name syntax. For instance:

In [6]:
nest.get('map') is nest.db('P_map') # the get method accesses the P_map symbols

True

In [7]:
nest.get('map', local = 'W') is nest.db('W_map') # if we specify local, we access symbols from the individual tree

True

*Note: The notebook ```nestingTree.ipynb``` under the gmsPython package details what this nesting tree class does in more detail.*

### 1.2. Create value shares to identify $\mu$

The next step is to identify $\mu$ parameters for the nested model. We base this on the value shares in the baseline year. For instance, if we know the value of energy and capital in baseline $vD(E), vD(K)$, a CES nest that combines the two will have $\mu_E = vD(E)/(vD(E)+vD(K))$ and $\mu_K = vD(K)/(vD(E)+vD(K))$. To do this with the more general nesting structure, we use a simple class ```nestedShares``` that ultimately writes a GAMS code and let GAMS solve it:

We initialize a ```nestedShares``` class using a nesting tree (note that, if possible, we also always use an existing gams workspace (ws) to start new databases/models from):

In [8]:
v = nestedShares(nest, ws = ws)

*Note: The model is initialized and solve using the following one-liner. To explain what happens in a bit more detail, we have commented this out and replaced with text and low-level functions.*

In [9]:
# db_vs = v(db) # one-liner that returns solution database

This simple class then runs through the following steps:
* Initialize relevant data (this creates some subsets/mappings/initial values for the GAMS model) by drawing on the IO database.
* Use ```mergeInternal``` method from the database to write a gdx file with relevant data (before doing this, we only have data in Python format).
* Initialize group definitions (what symbols are endogenous, what symbols are exogenous).
* Next, we call the ```self.text``` property which essentially writes the entire GAMS code. Below, we break down the different parts in a bit more detail here.
* We run the text through the ```gmsPython.gamY.Precompiler``` and start a ```GamsJob```. We then extract the solution to Python again using the ```pyDatabases.GpyDB``` database format.

*Note: All of these steps are carried out in one step using the ```self.__call__(db)``` method*.

In [10]:
v.initData(db) # writes data to the v.db database
v.db.mergeInternal() # writes gdx file with relevant data 
v.initGroups() # initialize grouping of variables

Let us have a look at the groups that are added as an attribute as ```self.groups```. We add two simple groups 'endo' and 'exo':

In [11]:
v.groups

{'endo': <gmsPython.gmsPy.gmsPy.Group at 0x1d5f3f51850>,
 'exo': <gmsPython.gmsPy.gmsPy.Group at 0x1d5f738a910>}

The group 'endo' contains shares $\mu$ and values $vD$ for intermediate symbols in the nesting tree. We specify that it is only $vD$ for intermediate goods that are endogenous by adding a condition to the statement for $vD$:

In [12]:
v.groups['endo'].conditions

{'mu': <pyDatabases.gpyDB.database.gpy at 0x1d5f738b790>,
 'vD': <pyDatabases.gpyDB.database.gpy at 0x1d5f738b7d0>}

Note that the condition is actually a ```gpy``` symbol (the ones we use in our databases). We could also simply have written the text here, that is "input[s,n]". By referring to the ```gpy``` symbol instead, we can extract the relevant symbol in Python as well if need be:

In [13]:
adj.rc_pd(v.db('vD'), v.db['input']); # this returns a pd.Series with vD only for the symbol.

The groups have buildin writing methods, so we can call e.g. the ```unfix``` method get the text required to make variables endogenous (or the ```fix``` method to make them exogenous):

In [14]:
print(v.groups['endo'].unfix(db = v.db)) # return text to make sure that variables are endogenous (no bounds)

mu.lo[t,s,n,nn]$(map[s,n,nn]) = -inf;
mu.up[t,s,n,nn]$(map[s,n,nn]) = inf;
vD.lo[t,s,n]$(int[s,n]) = -inf;
vD.up[t,s,n]$(int[s,n]) = inf;


In this simple model, the gamY code consists of 7 steps:
1. Root text (```gmsWrite.StdArgs.root(**options)```): Returns a text that declares some standard options.
2. Declare functions and macros (```gmsWrite.StdArgs.funcs(**kwargs)```): Returns text that declares standard functions (gamY feature). I have not implemented adjustments to this yet.
3. Declare symbols (```gmsWrite.FromDB.declare(db)```): Returns text declaring all symbols in a database.
4. Load symbols (```gmsWrite.FromDB.load(db, gdx)```): Returns text that loads symbols from a database with name specified by argument ```gdx```.
5. Define blocks of equations: No automated feature here.
6. Fix/unfix relevant variables: This is automated by the ```gmsPython.Group``` instances above.
7. Solve statement.

In [15]:
print(v.text) # the GAMS model



$SETLOCAL qmark ";
OPTION SYSOUT=OFF, SOLPRINT=OFF, LIMROW=0, LIMCOL=0, DECIMALS=6;


# User defined functions:
$FUNCTION SolveEmptyNLP({name})
variable randomnameobj;  
randomnameobj.L = 0;

EQUATION E_EmptyNLPObj;
E_EmptyNLPObj..    randomnameobj  =E=  0;

Model M_SolveEmptyNLP /
E_EmptyNLPObj, {name}
/;
solve M_SolveEmptyNLP using NLP min randomnameobj;
$ENDFUNCTION

sets
	alias_set
	alias_map2
	s
	n
	t
;

alias(n,nn);

sets
	alias_[alias_set,alias_map2]
	mapOut[s,n,nn]
	knotOutTree[s,n]
	branchOut[s,n]
	branchNOut[s,n]
	mapInp[s,n,nn]
	knotOut[s,n]
	knotNOut[s,n]
	branch2Out[s,n]
	branch2NOut[s,n]
	map[s,n,nn]
	output[s,n]
	input[s,n]
	int[s,n]
;

variables
	mu[t,s,n,nn]
	vD[t,s,n]
	vS[t,s,n]
;


$GDXIN db
$onMulti
$load alias_set
$load alias_map2
$load s
$load n
$load t
$load alias_
$load mapOut
$load knotOutTree
$load branchOut
$load branchNOut
$load mapInp
$load knotOut
$load knotNOut
$load branch2Out
$load branch2NOut
$load map
$load output
$load input
$load int
$GDXIN
$offMu

The gamY code is then translated to GAMS code by running it through the ```gmsPython.gamY.Precompiler``` that is attached to the ```nestedShares``` as an attribute:

In [16]:
GAMStext = v.compiler(v.text) # return GAMS code

We execute GAMS code by initializing a GamsJob and running it as follows:

In [17]:
v.job = v.ws.add_job_from_string(GAMStext)
v.job.run(databases = v.db.database)
db_vs = GpyDB(v.job.out_db, ws = v.ws) # extract solution

Everything related to the GAMS code, gdx files, etc. are written to the work folder (specified at ```v.work_folder```). 

We are particularly interested in the $\mu$ variable and value of intermediate goods (to get better initial values):

In [18]:
db['mu'] = db_vs('mu').xs(t0) 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

With the share parameters in place, we can turn to the production module. We start by making a few small adjustments to the IO data, so we can use this as an outset for the module.

## 2. Partial Equilibirum Model

In [19]:
db.aom(db('pD_dur'), name = 'pD')
AggDB.subsetDB(db, db('s_p'))

Clean up database a bit (this is not necessary, but it removes some variables that are not ultimately used in the model):

In [20]:
[db.series.__delitem__(k) for k in ('vD','vTax', 'vD_dur','vD_depr','vD_inv', 'vS', 'pD_dur') if k in db.symbols];

For variables that are defined over $t$, but we do not yet have an initial value for all $t$, extrapolate:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [21]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

Next, we can now initialize the production module using from the nesting tree, we established earlier. 

In [22]:
m = NestedCES(nest, partial = True)

Next, we initialize stuff (add data from IO, add specification durables, fill in initial values of data where we do not have any from the IO data, initialize grouping of variables):

In [23]:
m.initStuff(db = db)

The production module is, numerically, quite a challenging problem to solve. To solve and calibrate the model, we use a fairly robust solution method that solves a series of problems that converges on the true problem. Essentially, we do the following:
1. Write GAMS model without solving it initially.
2. Create a copy of the GAMS model, but add "$j$-terms" to every equation in the model (additive term that captures deviation from actual equilibrium).
3. Fix the value of all variables *except* $j$-terms. "Solve" the adjusted model (solution where $j$-terms simply measure deviation from eq. in initial point).
4. Now, fix all the $j$-terms (turn into parameters) and unfix the original endogenous variables in the model. The model is now solved in initial point as well.
5. Define a series of $j$-terms that converges to $0$ (in which case the model coincides with original formulation again). The more "numerically tricky" the problem is, the slower convergence is required for the solver to work.
6. Once the series of $j$-terms have converged to zero, we have a baseline solution to the model.
   

The smart thing about this approach is that it is *generic*, i.e. independent of the type of problem that arises (as long as the problem stays feasible along the convering path). This means that we can easily automate this solution procedure and use not only for the production module, but all the other modules/models as well. This type of converging series of solutions is implemented through the class ```gmsPython.gmsPy.jTerms``` which is automatically attached to model instances like the production module:

In [24]:
soldb = m.jSolve(10, state = 'C', ϕ = .1) # solve calibration model with 10 steps and nonlinear grid (ϕ<1 means that adjustments to jTerms start large and then decrease)

Given this, the model should be solved in the initial point with the baseline model as well:

In [25]:
[m.db.__setitem__(k, soldb[k]) for k in m.db.getTypes(['var']) if k in soldb.symbols]; # use solution database
m.db.mergeInternal()

Baseline model text with fix/unfix and solve statements:

In [26]:
baselineText = m.write(state = 'B') # store text so we can export it for later
text_gamY    = m.write_gamY(state = 'B') # store gamY text
m.solve(text = baselineText)

Export code and database:

In [27]:
with open(os.path.join(d['gams'], f'{m.name}.gms'), "w") as file:
    file.write(baselineText)
with open(os.path.join(d['gams'], f'{m.name}.gmy'), "w") as file:
    file.write(text_gamY)

Export gdx file:

In [28]:
m.db.database.export(os.path.join(d['data'], m.db.name))